In [1]:
#Problem1:
#Building a spam classification engine.
#Data: SMSSpamCollection.csv (Manually tagged data with 5573 messages)
#Follow the steps given below and build code for classification engine and evaluate the model:

In [2]:
# import the required libraries
import numpy as np
import pandas as pd
import os
import re

In [8]:
# Read the data into working envrionment
data = pd.read_csv("SMSSpamCollection.csv", delimiter="\t")# Reading the data
data.head(5)
data.columns = ['Type','Text'] # Changing column names
type(data)# type is dataframe
data.head(5)

,Type,Text
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [12]:
# Text preprocessing
Text_data = data['Text']
type(data) # Type is dataframe
Text_data1 = Text_data.tolist()
#Some text preprocessing- Removing extra spaces and numbers
for i in range(len(Text_data1)):
    Text_data1[i]=re.sub(r'\s+',' ',Text_data1[i]) #Removing more than one whfroite spaces in the sentense
    Text_data1[i]= re.sub('[\d]','',Text_data1[i]) # Removing the digits
# Check the output
Text_data1[8]


'Had your mobile  months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on '

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Text_data1, data['Type'],test_size=0.3,random_state=1234)
#Ensure similar distribution of target classes in both train and test
y_train.value_counts()
y_test.value_counts()

ham     1446
spam     226
Name: Type, dtype: int64

In [15]:
#Ensure similar distribution of target classes in both train and test
y_train.value_counts()

ham     3378
spam     521
Name: Type, dtype: int64

In [ ]:
#??tfidf_transformer.fit #"""Learn vocabulary and idf from training set.
#??tfidf_transformer.transform # """Transform documents to document-term matrix.
#Uses the vocabulary and document frequencies (df) learned by fit (or fit_transform)

In [21]:
# Construct tf-idf matrix
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
#Lets construct TF-idf matrix based on the train documents
tfidf_transformer = TfidfVectorizer(ngram_range=(1,1),stop_words='english')
X_train_tfidf = tfidf_transformer.fit_transform(X_train)
X_train_tfidf.shape
# Get the tfidf matrix for test documents
X_test_tfidf = tfidf_transformer.transform(X_test)
X_test_tfidf.shape

(1672, 6206)

In [22]:
#Implement Naive Bayes Classification model
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB().fit(X_train_tfidf, y_train)#nb_clf is the classfier
pred_train = nb_clf.predict(X_train_tfidf) # get predictions on train data

In [25]:
from sklearn.metrics import confusion_matrix
prediction=confusion_matrix(y_train, pred_train) #get confusion matrix for train predictions
# Metrics

print("Train_Confusion Matrix:\n",prediction,"\n")

from sklearn.metrics import recall_score, precision_score, accuracy_score
acc = accuracy_score(y_train,pred_train)
rec = recall_score(y_train, pred_train, pos_label='spam')
prec = precision_score(y_train,pred_train,pos_label='spam')
print("Results of Naive Bayes on train data:","\nAccuracy:",acc, "\nRecall:",rec,
"\nPrecision:",prec)

Train_Confusion Matrix:
 [[3378    0]
 [  71  450]] 

Results of Naive Bayes on train data: 
Accuracy: 0.9817902026160554 
Recall: 0.8637236084452975 
Precision: 1.0


In [29]:
# Analyze model performance on test data
pred = nb_clf.predict(X_test_tfidf) # predict on test data 
prediction = confusion_matrix(y_test, pred) # Getthe confusion matrix for test_predictions
print("Test_Confusion_matrix:\n", prediction,"\n")
acc = accuracy_score(y_test,pred)
rec = recall_score(y_test,pred, pos_label='spam')
prec = precision_score(y_test, pred, pos_label='spam')
print("Results of Naive Bayes on test data:", "\nAccuracy:", acc, "\nRecall:", rec, "\nPrecision",prec)

Test_Confusion_matrix:
 [[1445    1]
 [  56  170]] 

Results of Naive Bayes on test data: 
Accuracy: 0.9659090909090909 
Recall: 0.7522123893805309 
Precision 0.9941520467836257


In [36]:
from sklearn import linear_model
logreg_train = linear_model.LogisticRegression()
lr_clf = logreg_train.fit(X_train_tfidf,y_train)# fit the model
pred_lr_train=lr_clf.predict(X_train_tfidf) # predict on train data
pred_lr_test=lr_clf.predict(X_test_tfidf) # predict on test data


print("Train_Confusion matrix:\n", confusion_matrix(y_train,pred_lr_train))
print("Test_Confusion matrix:\n",confusion_matrix(y_test,pred_lr_test))

Train_Confusion matrix:
 [[3375    3]
 [ 128  393]]
Test_Confusion matrix:
 [[1443    3]
 [  72  154]]


In [38]:
print("Results of Logistic regression on test data")
acc=accuracy_score(y_test,pred_lr_test)
rec=recall_score(y_test,pred_lr_test,pos_label='spam')
prec=precision_score(y_test,pred,pos_label='spam')
print("\nAcc: ",acc,"\nRec:", rec, "\nPrec:", prec)

Results of Logistic regression on test data

Acc:  0.9551435406698564 
Rec: 0.6814159292035398 
Prec: 0.9941520467836257
